## Section 1: Setup and Data Loading

In [ ]:
import os
import sys

# Find project root (cv-job-matcher-project/)
# Notebooks are at */notebooks/ so we need to go up TWO levels
cwd = os.getcwd()
if 'notebooks' in cwd:
    PROJECT_ROOT = os.path.dirname(os.path.dirname(cwd))  # TWO levels up
else:
    PROJECT_ROOT = cwd
sys.path.insert(0, PROJECT_ROOT)

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
# Setup working directory
import os
os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

CV_DATA_PATH = os.path.join(PROJECT_ROOT, 'ingest_cv', 'output', 'cv_query_text.parquet')
JOBS_EMB_PATH = os.path.join(PROJECT_ROOT, 'training', 'output', 'embeddings', 'jobs_embedded.parquet')

print(f"CV data exists: {os.path.exists(CV_DATA_PATH)}")
print(f"Jobs data exists: {os.path.exists(JOBS_EMB_PATH)}")

In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sentence_transformers import CrossEncoder, SentenceTransformer
import faiss
import os
from tqdm import tqdm

print("Imports successful")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

Imports successful
Pandas version: 3.0.0
Numpy version: 1.26.4


In [ ]:
#load CV query texts (7299 CVs with Query: prefix)
cv_df = pd.read_parquet(CV_DATA_PATH)
print(f"Loaded CV data: {cv_df.shape}")
print(f"Columns: {cv_df.columns.tolist()}")
print(f"First CV ID: {cv_df.iloc[0]['id']}")
print(f"First CV text (first 400 chars): {cv_df.iloc[0]['text'][:400]}")

In [ ]:
#load job embeddings (165K jobs with passage: prefix)
jobs_df = pd.read_parquet(JOBS_EMB_PATH)
print(f"Loaded jobs data: {jobs_df.shape}")
print(f"Columns: {jobs_df.columns.tolist()}")
print(f"First job ID: {jobs_df.iloc[0]['job_id']}")
print(f"First job text (first 400 chars): {jobs_df.iloc[0]['embedding_text'][:400]}")
print(f"Embedding shape: {jobs_df.iloc[0]['embedding'].shape if hasattr(jobs_df.iloc[0]['embedding'], 'shape') else 'N/A'}")

In [ ]:
# load train/val/test split IDs
CV_SPLITS_PATH = os.path.join(PROJECT_ROOT, 'ingest_cv', 'output')
train_ids = pd.read_parquet(os.path.join(CV_SPLITS_PATH, 'training_set_cv_ids.parquet'))
val_ids = pd.read_parquet(os.path.join(CV_SPLITS_PATH, 'validation_set_cv_ids.parquet'))
test_ids = pd.read_parquet(os.path.join(CV_SPLITS_PATH, 'test_set_cv_ids.parquet'))

print(f"Training set: {len(train_ids)} CVs")
print(f"Validation set: {len(val_ids)} CVs")
print(f"Test set: {len(test_ids)} CVs")
print(f"Total: {len(train_ids) + len(val_ids) + len(test_ids)} CVs")
print()
print(f"Sample training IDs: {train_ids['anchor'].iloc[:5].tolist()}")
print(f"Sample validation IDs: {val_ids['anchor'].iloc[:5].tolist()}")

In [6]:
# !!! fix CV prefix from "Query: " to "query: " (lowercase)
# This must happen BEFORE any encoding for e5-base-v2
cv_df['text_fixed'] = cv_df['text'].apply(lambda x: x.replace("Query: ", "query: "))

# Verify fix
print(f"Original prefix: {cv_df.iloc[0]['text'][:50]}")
print(f"Fixed prefix: {cv_df.iloc[0]['text_fixed'][:50]}")
assert cv_df.iloc[0]['text_fixed'].startswith("query: "), "ERROR: Prefix fix failed!"
print("Prefix fix VERIFIED")
print()

Original prefix: Query: I am a Python Developer with 0 years of exp
Fixed prefix: query: I am a Python Developer with 0 years of exp
Prefix fix VERIFIED



In [7]:
# Extract job embeddings and build Faiss index

job_embeddings = np.array(jobs_df['embedding'].tolist()).astype('float32')
print(f"Job embeddings shape: {job_embeddings.shape}")

# Normalize embeddings for cosine similarity (IndexFlatIP)
faiss.normalize_L2(job_embeddings)
print("Embeddings normalized")

# Build index
dimension = job_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(job_embeddings)
print(f"Index built with {index.ntotal} jobs")

Job embeddings shape: (165193, 768)
Embeddings normalized
Index built with 165193 jobs


In [8]:
# Load e5-base-v2 model to encode CVs
bi_encoder = SentenceTransformer('intfloat/e5-base-v2')
print("e5-base-v2 model for CV encoding loaded")

e5-base-v2 model for CV encoding loaded


In [9]:
def generate_candidate_pairs(cv_ids_df, cv_data, top_k=50):
    # generate candidate pairs for given CV IDs
    #returns list of (cv_id, job_id) tuples

    print(f"Generating candidates for {len(cv_ids_df)} CVs (top-{top_k} per CV)")
    
    # filter CVs to only those in the split
    cv_subset = cv_data[cv_data['id'].isin(cv_ids_df['anchor'])].copy()
    print(f"Filtered to {len(cv_subset)} CVs")
    
    # encode CV texts (with fixed lowercase prefix)
    cv_texts = cv_subset['text_fixed'].tolist()
    print(f"encoding {len(cv_texts)} CV texts")
    cv_embeddings = bi_encoder.encode(cv_texts, show_progress_bar=True, convert_to_numpy=True)
    print(f"CV embeddings shape: {cv_embeddings.shape}")
    
    # normalize for cosine similarity
    cv_embeddings = cv_embeddings.astype('float32')
    faiss.normalize_L2(cv_embeddings)
    print("CV embeddings normalized")
    
    # search for top-k jobs per CV
    print(f"searching for top-{top_k} jobs per CV")
    scores, indices = index.search(cv_embeddings, top_k)
    print(f"Search complete: {scores.shape}")
    
    # build candidate pairs
    pairs = []
    for i, cv_id in enumerate(cv_subset['id']):
        for j in range(top_k):
            job_idx = indices[i][j]
            job_id = jobs_df.iloc[job_idx]['job_id']
            pairs.append((cv_id, job_id))
    
    print(f"Generated {len(pairs)} candidate pairs")
    return pairs

print("Function defined")

Function defined


In [10]:
# generate training candidates
print("TRAINING SET")
training_candidates = generate_candidate_pairs(train_ids, cv_df, top_k=50)
print(f"Total training candidates: {len(training_candidates)}")
print(f"Expected: {len(train_ids)} CVs * 50 = {len(train_ids) * 50}")

TRAINING SET
Generating candidates for 5839 CVs (top-50 per CV)
Filtered to 5839 CVs
encoding 5839 CV texts


Batches:   0%|          | 0/183 [00:00<?, ?it/s]

CV embeddings shape: (5839, 768)
CV embeddings normalized
searching for top-50 jobs per CV
Search complete: (5839, 50)
Generated 291950 candidate pairs
Total training candidates: 291950
Expected: 5839 CVs * 50 = 291950


In [11]:
# generate validation candidates
print("validation set")
validation_candidates = generate_candidate_pairs(val_ids, cv_df, top_k=50)
print(f"Total validation candidates: {len(validation_candidates)}")
print(f"Expected: {len(val_ids)} CVs * 50 = {len(val_ids) * 50}")

validation set
Generating candidates for 730 CVs (top-50 per CV)
Filtered to 730 CVs
encoding 730 CV texts


Batches:   0%|          | 0/23 [00:00<?, ?it/s]

CV embeddings shape: (730, 768)
CV embeddings normalized
searching for top-50 jobs per CV
Search complete: (730, 50)
Generated 36500 candidate pairs
Total validation candidates: 36500
Expected: 730 CVs * 50 = 36500


In [12]:
# Load cross-encoder model
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L12-v2')
print("Cross-encoder (ms-marco-MiniLM-L12-v2) loaded")

Cross-encoder (ms-marco-MiniLM-L12-v2) loaded


In [13]:
def score_pairs_with_cross_encoder(candidates, cv_data, job_data, batch_size=128):
    #score candidate pairs with cross-encoder
    # returns DataFrame with cv_id, job_id, score
    print(f"Scoring {len(candidates)} pairs with cross-encoder")
    
    # build lookup dicts for fast access
    cv_text_map = dict(zip(cv_data['id'], cv_data['text_fixed']))
    job_text_map = dict(zip(job_data['job_id'], job_data['embedding_text']))
    
    # prepare text pairs for cross-encoder
    # !!!! Cross-encoder does NOT use prefixes, so we strip them
    text_pairs = []
    for cv_id, job_id in candidates:
        cv_text = cv_text_map[cv_id]
        job_text = job_text_map[job_id]
        
        # strip prefixes
        cv_text_clean = cv_text.replace("query: ", "")
        job_text_clean = job_text.replace("passage: ", "")
        
        text_pairs.append([cv_text_clean, job_text_clean])
    
    print(f"Prepared {len(text_pairs)} text pairs")
    print(f"Sample pair:")
    print(f" CV (first 300 chars): {text_pairs[0][0][:300]}")
    print(f" Job (first 300 chars): {text_pairs[0][1][:300]}")
    
    # score in batches
    print(f"Scoring in batches of {batch_size}")
    scores = cross_encoder.predict(text_pairs, batch_size=batch_size, show_progress_bar=True)
    print(f"Scored {len(scores)} pairs")
    
    # build results DataFrame
    results = pd.DataFrame({
        'cv_id': [pair[0] for pair in candidates],
        'job_id': [pair[1] for pair in candidates],
        'score': scores
    })
    
    print(f"Results shape: {results.shape}")
    print(f"Score stats: min={results['score'].min():.4f}, max={results['score'].max():.4f}, mean={results['score'].mean():.4f}")
    
    return results

print("Function defined")

Function defined


In [14]:
# score training pairs
print("SCORING TRAINING PAIRS")
training_scores_df = score_pairs_with_cross_encoder(training_candidates, cv_df, jobs_df, batch_size=128)
print(f"Training scores complete: {training_scores_df.shape}")

SCORING TRAINING PAIRS
Scoring 291950 pairs with cross-encoder
Prepared 291950 text pairs
Sample pair:
 CV (first 300 chars): I am a DevOps Engineer with 0 years of experience, (internship level). My skills include: jquery, Xillinx, English, Javascript, mysql, HTML, Modelsim, Spring, JAVA, SQL, JAVAEE, PL/SQL, Python, Hibernate, Multisim, Matlab, AWS, C, CSS. I worked as DevOps Engineer from 2026. I completed my B.E. at Sa
 Job (first 300 chars): Role of Software Development Engineer IV- #: 23-19983 at TalentBurst, an Inc 5000 company in Berkeley Heights, NJ. Required skills: DevOps, Java, Jbuzz, Azure, Cloud Computing, Linux, Shell Scripting, Splunk, Troubleshooting, Disaster Recovery. Experience level: Mid-level, 3-5 years experience.
Scoring in batches of 128


Batches:   0%|          | 0/2281 [00:00<?, ?it/s]

Scored 291950 pairs
Results shape: (291950, 3)
Score stats: min=-9.8367, max=8.1766, mean=3.6931
Training scores complete: (291950, 3)


In [15]:
# score validation pairs
print("SCORING VALIDATION PAIRS")
validation_scores_df = score_pairs_with_cross_encoder(validation_candidates, cv_df, jobs_df, batch_size=128)
print(f"Validation scores complete: {validation_scores_df.shape}")

SCORING VALIDATION PAIRS
Scoring 36500 pairs with cross-encoder
Prepared 36500 text pairs
Sample pair:
 CV (first 300 chars): I am a Python Developer with 0 years of experience, (internship level). My skills include: MySQL, C++, Python, Windows, Fedora, Ubuntu, C, Cent OS, Tensorflow, Numpy. I worked as Python Developer from 2026 (6 months). I completed my ME Computer Engineering at Savitribai Phule Pune University around 
 Job (first 300 chars): Role of 🌟 Expression of Interest - Python Engineering Opportunities 🐍 at Hyre. in Sydney, New South Wales, Australia. Required skills: Python, Machine learning, Data analytics, Data visualization, Predictive modeling, Data pipelines, Data requirements, Datadriven solutions, Software frameworks, Data
Scoring in batches of 128


Batches:   0%|          | 0/286 [00:00<?, ?it/s]

Scored 36500 pairs
Results shape: (36500, 3)
Score stats: min=-9.6652, max=8.0829, mean=2.5920
Validation scores complete: (36500, 3)


In [16]:
def select_best_matches(scores_df):
    # for each CV, select the job with highest score
    # returns DataFrame with anchor (CV), match (Job)

    print(f"Selecting best matches from {len(scores_df)} scored pairs")
    
    #group by CV and select argmax
    best_matches = scores_df.loc[scores_df.groupby('cv_id')['score'].idxmax()]
    print(f"Selected {len(best_matches)} best matches")
    
    #rrename columns to match expected format
    result = best_matches[['cv_id', 'job_id', 'score']].copy()
    result.columns = ['anchor', 'match', 'score']
    
    print(f"Result shape: {result.shape}")
    print(f"Best match score stats: min={result['score'].min():.4f}, max={result['score'].max():.4f}, mean={result['score'].mean():.4f}")
    
    return result

print("Function defined")

Function defined


In [17]:
#select best training matches
print("TRAINING BEST MATCHES")
training_pairs = select_best_matches(training_scores_df)
print(f"Training pairs: {len(training_pairs)}")
print(f"Expected: {len(train_ids)} (one per CV)")
print(f"Sample pairs:")
print(training_pairs.head(10))

TRAINING BEST MATCHES
Selecting best matches from 291950 scored pairs
Selected 5839 best matches
Result shape: (5839, 3)
Best match score stats: min=-8.4054, max=8.1766, mean=4.9274
Training pairs: 5839
Expected: 5839 (one per CV)
Sample pairs:
      anchor         match     score
25954  A1000        B24576  7.175061
26000  A1001  B17179921701  6.185480
26091  A1002  B60129626521  6.125921
26139  A1005  B68719501193  7.130793
26198  A1006  B25769888661  6.749949
26204  A1007  B51539630226  5.285763
26250  A1008  B17179921701  6.950738
26311  A1009  B17179893677  6.437855
1990    A101  B85899363744  7.434629
26354  A1011        B84872  6.760271


In [18]:
#best validation matches
print("VALIDATION BEST MATCHES")
validation_pairs = select_best_matches(validation_scores_df)
print(f"Validation pairs: {len(validation_pairs)}")
print(f"Expected: {len(val_ids)} (one per CV)")
print(f"Sample pairs:")
print(validation_pairs.head(10))

VALIDATION BEST MATCHES
Selecting best matches from 36500 scored pairs
Selected 730 best matches
Result shape: (730, 3)
Best match score stats: min=-6.1732, max=8.0829, mean=4.0498
Validation pairs: 730
Expected: 730 (one per CV)
Sample pairs:
      anchor         match     score
13        A1  B42949739960  4.546541
1004    A100  B42949739960  6.095415
1054    A105  B42949739960  6.095416
1105    A108  B34359826064  5.323551
13208  A1094  B42949756750  6.388457
13279  A1106  B77309429076  6.746703
13325  A1132  B85899418295  7.132197
1195    A116  B68719495460  6.879686
13353  A1205  B85899418295  4.711871
1200    A121  B51539675205  5.993254


In [19]:
#output directory
output_dir = 'output/training_data'
os.makedirs(output_dir, exist_ok=True)
print(f"Created output directory: {output_dir}")

Created output directory: output/training_data


In [20]:
# save training pairs (ID only)
training_pairs_to_save = training_pairs[['anchor', 'match']].copy()
training_pairs_to_save.to_parquet(f'{output_dir}/training_pairs.parquet', index=False)
training_pairs_to_save.to_csv(f'{output_dir}/training_pairs.csv', index=False)
print(f"Saved training pairs: {len(training_pairs_to_save)} rows")
print(f" - {output_dir}/training_pairs.parquet")
print(f" - {output_dir}/training_pairs.csv")

Saved training pairs: 5839 rows
 - output/training_data/training_pairs.parquet
 - output/training_data/training_pairs.csv


In [21]:
# save validation pairs (ID only)
validation_pairs_to_save = validation_pairs[['anchor', 'match']].copy()
validation_pairs_to_save.to_parquet(f'{output_dir}/validation_pairs.parquet', index=False)
validation_pairs_to_save.to_csv(f'{output_dir}/validation_pairs.csv', index=False)
print(f"Saved validation pairs: {len(validation_pairs_to_save)} rows")
print(f" - {output_dir}/validation_pairs.parquet")
print(f" - {output_dir}/validation_pairs.csv")

Saved validation pairs: 730 rows
 - output/training_data/validation_pairs.parquet
 - output/training_data/validation_pairs.csv


In [22]:
# save training text data for bi-encoder

#build CV text lookup
cv_text_map = dict(zip(cv_df['id'], cv_df['text_fixed']))
job_text_map = dict(zip(jobs_df['job_id'], jobs_df['embedding_text']))
#add text to training pairs
training_dataset = training_pairs.copy()
training_dataset['anchor_text'] = training_dataset['anchor'].map(cv_text_map)
training_dataset['positive_text'] = training_dataset['match'].map(job_text_map)
# save
training_dataset[['anchor_text', 'positive_text']].to_parquet(f'{output_dir}/training_dataset.parquet', index=False)
print(f"Saved training dataset: {len(training_dataset)} rows")
print(f" - {output_dir}/training_dataset.parquet")
print(f" Columns: {training_dataset.columns.tolist()}")

Saved training dataset: 5839 rows
 - output/training_data/training_dataset.parquet
 Columns: ['anchor', 'match', 'score', 'anchor_text', 'positive_text']


In [23]:
#save validation text data for bi-encoder

#add text to validation pairs
validation_dataset = validation_pairs.copy()
validation_dataset['anchor_text'] = validation_dataset['anchor'].map(cv_text_map)
validation_dataset['positive_text'] = validation_dataset['match'].map(job_text_map)
# save
validation_dataset[['anchor_text', 'positive_text']].to_parquet(f'{output_dir}/validation_dataset.parquet', index=False)
print(f"Saved validation dataset: {len(validation_dataset)} rows")
print(f" - {output_dir}/validation_dataset.parquet")
print(f" Columns: {validation_dataset.columns.tolist()}")

Saved validation dataset: 730 rows
 - output/training_data/validation_dataset.parquet
 Columns: ['anchor', 'match', 'score', 'anchor_text', 'positive_text']


In [24]:
# Print file sizes
import os
print("OUTPUT FILES")

for filename in os.listdir(output_dir):
    filepath = os.path.join(output_dir, filename)
    size_mb = os.path.getsize(filepath) / (1024 * 1024)
    print(f"{filename}: {size_mb:.2f} MB")

print("\nTraining data preparation complete")

OUTPUT FILES
training_dataset.parquet: 1.21 MB
training_pairs.csv: 0.10 MB
validation_dataset.parquet: 0.22 MB
validation_pairs.csv: 0.01 MB
validation_pairs.parquet: 0.01 MB
training_pairs.parquet: 0.06 MB

Training data preparation complete


In [25]:
# verification Cell 1: Prefix Check
# verify CV prefix standardization
sample_cv = training_dataset.iloc[0]['anchor_text']
print(f"Sample CV text (first 200 chars): {sample_cv[:200]}")
assert sample_cv.startswith("query: "), "ERROR: CV text should start with lowercase 'query: '"
print("✓ CV prefix check PASSED")

# verify job prefix
sample_job = training_dataset.iloc[0]['positive_text']
print(f"Sample job text (first 200 chars): {sample_job[:200]}")
assert sample_job.startswith("passage: "), "ERROR: Job text should start with 'passage: '"
print("✓ Job prefix check PASSED")

Sample CV text (first 200 chars): query: I am a Devops Engineer with 3 years of experience, (mid-level). My skills include: GitLab, C++, English, Go, REST, Google Cloud, fluent, MongoDB, AWS, Agile, GraphQL, PostgreSQL, Jenkins, CI/CD
✓ CV prefix check PASSED
Sample job text (first 200 chars): passage: Role of DevOps/Software Engineer III at Steneral Consulting in Houston, TX. Required skills: Python, CI/CD, Azure DevOps, Jenkins, GitLab, Kubernetes, Helm, Ansible, Chef, Puppet. Experience 
✓ Job prefix check PASSED


In [26]:
#verification Cell 2: ID Format Check
print("VERIFICATION 2: ID FORMAT CHECK")

import re

# check CV IDs (should be A followed by number)
cv_id_pattern = r'^A\d+$'
invalid_cv_ids = training_pairs[~training_pairs['anchor'].str.match(cv_id_pattern)]
print(f"Invalid CV IDs: {len(invalid_cv_ids)}")
assert len(invalid_cv_ids) == 0, f"Found invalid CV IDs: {invalid_cv_ids['anchor'].tolist()[:5]}"
print("✓ CV ID format check PASSED")

# check Job IDs (should be B followed by number)
job_id_pattern = r'^B\d+$'
invalid_job_ids = training_pairs[~training_pairs['match'].str.match(job_id_pattern)]
print(f"Invalid Job IDs: {len(invalid_job_ids)}")
assert len(invalid_job_ids) == 0, f"Found invalid Job IDs: {invalid_job_ids['match'].tolist()[:5]}"
print("✓ Job ID format check PASSED")

VERIFICATION 2: ID FORMAT CHECK
Invalid CV IDs: 0
✓ CV ID format check PASSED
Invalid Job IDs: 0
✓ Job ID format check PASSED


In [ ]:
# verification Cell 3: Split Integrity
# verify no overlap between train/val/test
train_cvs = set(training_pairs['anchor'].values)
val_cvs = set(validation_pairs['anchor'].values)
test_cvs = set(pd.read_parquet(os.path.join(CV_SPLITS_PATH, 'test_set_cv_ids.parquet'))['anchor'].values)

train_val_overlap = train_cvs & val_cvs
train_test_overlap = train_cvs & test_cvs
val_test_overlap = val_cvs & test_cvs

print(f"Train set size: {len(train_cvs)}")
print(f"Val set size: {len(val_cvs)}")
print(f"Test set size: {len(test_cvs)}")
print()
print(f"Train-Val overlap: {len(train_val_overlap)}")
print(f"Train-Test overlap: {len(train_test_overlap)}")
print(f"Val-Test overlap: {len(val_test_overlap)}")

assert len(train_val_overlap) == 0, "ERROR: Train and validation sets overlap!"
assert len(train_test_overlap) == 0, "ERROR: Train and test sets overlap!"
assert len(val_test_overlap) == 0, "ERROR: Validation and test sets overlap!"
print("✓ Split integrity check PASSED")

In [28]:
# verification Cell 4: Score Statistics

# show cross-encoder score distribution for best matches
print("Training pairs score stats:")
print(f" Min: {training_pairs['score'].min():.4f}")
print(f" Max: {training_pairs['score'].max():.4f}")
print(f" Mean: {training_pairs['score'].mean():.4f}")
print(f" Median: {training_pairs['score'].median():.4f}")
print()

print("Validation pairs score stats:")
print(f" Min: {validation_pairs['score'].min():.4f}")
print(f" Max: {validation_pairs['score'].max():.4f}")
print(f" Mean: {validation_pairs['score'].mean():.4f}")
print(f" Median: {validation_pairs['score'].median():.4f}")

print("\nHigh scores indicate good CV-job matches")
print("✓ Score statistics computed")

Training pairs score stats:
 Min: -8.4054
 Max: 8.1766
 Mean: 4.9274
 Median: 5.8233

Validation pairs score stats:
 Min: -6.1732
 Max: 8.0829
 Mean: 4.0498
 Median: 5.1479

High scores indicate good CV-job matches
✓ Score statistics computed
